In [29]:
pip install requests pandas


Note: you may need to restart the kernel to use updated packages.


In [31]:
import requests
import pandas as pd

BASE_URL = "https://gamma-api.polymarket.com"
#curl "https://gamma-api.polymarket.com/events?series_id=10345&active=true&closed=false"


def get_nfl_series_id():
    url = f"{BASE_URL}/sports"
    response = requests.get(url)
    response.raise_for_status()
    sports = response.json()
    
    for league in sports:
        if league.get("sport", "").lower() == "nfl":
            return league["series"]   # IMPORTANT: use series
    
    raise ValueError("NFL not found")

def fetch_nfl_markets(series_id, limit=500):
    url = f"{BASE_URL}/events"
    
    params = {
        "series_id": series_id,
        "active": "true",
        "closed": "false",
        "limit": limit
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def normalize_market_data(events):
    rows = []
    
    for event in events:
        for market in event.get("markets", []):
            outcomes = market.get("outcomes", [])
            prices = market.get("outcomePrices", [])
            
            for outcome, price in zip(outcomes, prices):
                rows.append({
                    "event_id": event.get("id"),
                    "event_title": event.get("title"),
                    "market_id": market.get("id"),
                    "market_question": market.get("question"),
                    "outcome": outcome,
                    "implied_probability": float(price)
                })
    
    return pd.DataFrame(rows)

if __name__ == "__main__":
    
    print("Fetching NFL series ID...")
    nfl_series_id = get_nfl_series_id()
    print("NFL series_id:", nfl_series_id)
    
    print("Fetching NFL markets...")
    events = fetch_nfl_markets(nfl_series_id)
    
    print("Normalizing...")
    df = normalize_market_data(events)
    
    print(df.head())
    print("Total rows:", len(df))


Fetching NFL series ID...
NFL series_id: 10187
Fetching NFL markets...
Normalizing...
Empty DataFrame
Columns: []
Index: []
Total rows: 0


In [42]:
import requests
import pandas as pd

BASE_URL = "https://gamma-api.polymarket.com"

def fetch_all_markets(limit=500, max_pages=10):
    all_markets = []

    for page in range(max_pages):
        params = {
            "limit": limit,
            "offset": page * limit,
            "closed": "false"
        }

        response = requests.get(f"{BASE_URL}/markets", params=params)
        response.raise_for_status()
        markets = response.json()

        if not markets:
            break

        all_markets.extend(markets)

    return all_markets


def filter_super_bowl_futures(markets):
    sb_markets = []

    for m in markets:
        title = m.get("question", "").lower()

        if "super bowl" in title and "win" in title:
            sb_markets.append(m)

    return sb_markets


def extract_team_data(markets):
    rows = []

    for m in markets:
        outcomes = m.get("outcomes", [])
        prices = m.get("outcomePrices", [])
        volume = float(m.get("volume", 0))

        for outcome, price in zip(outcomes, prices):
            rows.append({
                "market_id": m.get("id"),
                "question": m.get("question"),
                "team": outcome,
                "implied_probability": float(price),
                "volume": volume
            })

    return pd.DataFrame(rows)


if __name__ == "__main__":

    print("Fetching markets...")
    markets = fetch_all_markets()

    print("Filtering Super Bowl futures...")
    sb_markets = filter_super_bowl_futures(markets)

    print("Markets found:", len(sb_markets))

    df = extract_team_data(sb_markets)

    print(df.head())
    print("Total rows:", len(df))


Fetching markets...
Filtering Super Bowl futures...
Markets found: 0
Empty DataFrame
Columns: []
Index: []
Total rows: 0


In [44]:
import requests
import pandas as pd

BASE_MARKETS = "https://gamma-api.polymarket.com/markets"
BASE_VOLUME = "https://data-api.polymarket.com/live-volume"

def fetch_all_markets(limit=500, max_pages=50):
    all_markets = []
    for page in range(max_pages):
        params = {
            "limit": limit,
            "offset": page * limit
        }
        r = requests.get(BASE_MARKETS, params=params)
        r.raise_for_status()
        markets = r.json()
        if not markets:
            break
        all_markets.extend(markets)
    return all_markets

def filter_super_bowl_markets(markets, year="2026"):
    sb_markets = []
    for m in markets:
        title = m.get("question", "").lower()
        if "super bowl" in title and year in title:
            sb_markets.append(m)
    return sb_markets

def get_market_volumes():
    r = requests.get(BASE_VOLUME)
    r.raise_for_status()
    return r.json()

def merge_volume(df, live_vol):
    vol_map = {}
    for entry in live_vol:
        for m in entry.get("markets", []):
            vol_map[m["market"]] = m["value"]
    df["live_volume"] = df["market_id"].map(vol_map)
    return df

def build_dataframe(sb_markets):
    rows = []
    for m in sb_markets:
        mid = m.get("id")
        for outcome, price in zip(m.get("outcomes", []), m.get("outcomePrices", [])):
            rows.append({
                "market_id": mid,
                "question": m.get("question"),
                "team_outcome": outcome,
                "implied_probability": float(price),
                "total_volume_indexed": float(m.get("volume", 0))
            })
    return pd.DataFrame(rows)

if __name__ == "__main__":
    print("Fetching all markets…")
    all_markets = fetch_all_markets()

    print(f"Total markets fetched: {len(all_markets)}")

    print("Filtering Gameday Super Bowl 60 markets…")
    sb_markets = filter_super_bowl_markets(all_markets, year="2026")
    print(f"Super Bowl markets found: {len(sb_markets)}")

    df = build_dataframe(sb_markets)

    print("Fetching live volume data…")
    live_vol = get_market_volumes()

    df = merge_volume(df, live_vol)

    print("Resulting DataFrame:")
    print(df.head())


Fetching all markets…
Total markets fetched: 25000
Filtering Gameday Super Bowl 60 markets…
Super Bowl markets found: 0
Fetching live volume data…


HTTPError: 400 Client Error: Bad Request for url: https://data-api.polymarket.com/live-volume